# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:

1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request
4. Create a basic AI agent
5. Handle basic error scenarios

# Prerequisites

- Completed environment setup from previous notebook
- Azure credentials configured

## Import Required Libraries and Setup

In the next cell, we'll:

1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard .Net libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
    - This will automatically use your logged-in Azure CLI credentials
    - Alternatively, it can use other authentication methods like environment variables or managed identity

In [ ]:
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System.IO;
using System.ClientModel.Primitives;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Azure.Core;
using OpenAI;
using OpenAI.Responses;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

TokenCredential CreateCredentialChainWithTenant()
{
    try
    {
        var cliCredentials = new AzureCliCredential(new AzureCliCredentialOptions{ TenantId = tenantId});
        var interactiveCredentials = new InteractiveBrowserCredential(new InteractiveBrowserCredentialOptions{TenantId = tenantId});
        var chainedCredentials = new ChainedTokenCredential(cliCredentials, interactiveCredentials);
        return chainedCredentials;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"⚠️ Credential chain creation error: {ex.Message}");
        return new DefaultAzureCredential(options: new DefaultAzureCredentialOptions{TenantId = tenantId}); 
    }
}

TokenCredential credential;
try
{
    credential = CreateCredentialChainWithTenant();
    var context = new TokenRequestContext(new[] { "https://management.azure.com/.default" });
    var testToken = credential.GetToken(context, default);
    Console.WriteLine("✅ Successfully initialized Azure credentials with correct tenant!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Failed to initialize Azure credentials: {ex.Message}");
    Console.WriteLine("💡 Please run the authentication fix from the environment setup notebook first");
}

## Initialize AI Project Client

**Note:** Before proceeding, ensure you:
1. Copy your `.env.example` file to `.env` from the root directory
2. Update the project endpoint in your `.env` file
3. Have a Foundry Project already provisioned in Azure AI Foundry

You can find your project endpoint in [Azure AI Foundry](https://ai.azure.com) under your project's settings:
<img src="../images/foundry-endpoint.png" alt="Project Endpoint Location" width="75%"/>

# Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our .env file.

    Note: This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing asyncio and using the async methods from AIProjectClient.

The client will be used to:

- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models

In [ ]:
var endpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT"); 
Console.WriteLine($"🔗 Project Endpoint: {endpoint}");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
Console.WriteLine($"🤖 Using Model Deployment Name: {modelDeploymentName}");

AIProjectClient projectClient;
try
{
    var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
    if (string.IsNullOrEmpty(projectEndpoint))
    {
        throw new InvalidOperationException("AI_FOUNDRY_PROJECT_ENDPOINT is not set in environment variables.");
    }
    Console.WriteLine($"🔗 Project Endpoint: {projectEndpoint}");
    
    projectClient = new AIProjectClient(new Uri(projectEndpoint), credential);
    Console.WriteLine("✅ Successfully initialized AI Project Client!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AI Project Client: {ex.Message}");
    Console.WriteLine("💡 Tip: Make sure your AI_FOUNDRY_PROJECT_ENDPOINT is set in the .env file");
    throw;
}

# Create a Simple Completion

Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request. The code below demonstrates how to:

- Get a OpeAI Responses client from the AI Project client
- Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our .env file, making it easy to switch between different deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers that support chat completions.

    Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in README.md)

In [ ]:
#pragma warning disable OPENAI001
try
{
    var chatClient = projectClient.GetProjectOpenAIClient().GetProjectResponsesClientForModel(modelDeploymentName);
    var question = "👤User: How to be healthy in one sentence?";
    var result = chatClient.CreateResponse(question);
    Console.WriteLine(question);
    var responseText = result?.Value.GetOutputText();
    Console.WriteLine($"🤖Assistant: {responseText}");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error during completion request: {ex.Message}");
    Console.WriteLine("💡 Tip: Ensure your MODEL_DEPLOYMENT_NAME is correct and the model is deployed.");
} 

# Create a simple Agent

Using Azure Foundry Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure Foundry Agent Service, a powerful tool for building intelligent agents.

Azure Foundry Agent Service is a fully managed service that helps developers build, deploy, and scale agents without managing infrastructure. It combines large language models with tools that allow agents to:

- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling
- Automate complex workflows

The code below demonstrates how to:

1. Create an agent with a code interpreter tool
3. Send a message requesting BMI analysis
4. Process the request and get results
5. Save any generated visualizations to local files

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access to a code interpreter tool for creating visualizations. This showcases how agents can combine natural language understanding with computational capabilities.

Note: Generated visualizations will be saved as PNG files in the same folder as this notebook.

In [ ]:
#pragma warning disable OPENAI001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

PromptAgentDefinition agentDefinition = new(model: modelDeploymentName)
{
    Instructions = @"You are a health analyst who calculates BMI using US metrics (pounds, feet/inches).
                     Use average US female measurements: 5'4\"" (69 inches) and 130 pounds.
                     Create a visualization showing where this BMI falls on the scale.",
    Tools = {
        ResponseTool.CreateCodeInterpreterTool(
            new CodeInterpreterToolContainer(
                CodeInterpreterToolContainerConfiguration.CreateAutomaticContainerConfiguration([])
            )
        ),
    }
};

AgentVersion agentVersion = await projectClient.Agents.CreateAgentVersionAsync(
    agentName: "bmi-calculator",
    options: new(agentDefinition));
AgentReference agentReference = new(name: agentVersion.Name, version: agentVersion.Version);
ProjectResponsesClient responseClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(agentReference);

ResponseResult response = await responseClient.CreateResponseAsync(@"
                  Calculate BMI for an average US female (5'4\"", 130 lbs). 
                  Create a visualization showing where this BMI falls on the standard BMI scale from 15 to 35.
                  Include the standard BMI categories (Underweight, Normal, Overweight, Obese) in the visualization.");

var containerClient = projectClient.OpenAI.GetContainerClient();
var lastResponse = response.OutputItems[response.OutputItems.Count - 1] as MessageResponseItem;
var responseText = lastResponse?.Content[0].Text;
Console.WriteLine($"🤖 Assistant:{responseText}");
var annotations = lastResponse?.Content[0].OutputTextAnnotations;
foreach(var item in annotations)
{
    var citationItem = item as ContainerFileCitationMessageAnnotation;
    try
    {
        var fileName = $"bmi_analysis_{citationItem.FileId}.png";
        var fileContent = await containerClient.DownloadContainerFileAsync(citationItem.ContainerId, citationItem.FileId);
        File.WriteAllBytes(fileName, fileContent.Value.ToArray());
        Console.WriteLine($"📊 Visualization saved as: {fileName}");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"⚠️ Could not save file {citationItem.FileId}: {ex.Message}");
        Console.WriteLine("💡 You can view the file in the Azure AI Foundry portal");
    }
}